Note that this project was inpired by the Wikipedia Crawler found here: https://github.com/hardikvasa/wikipedia-crawler
My project is more specific to Wikia pages (huge fandom-based website and the goal was to look at specific characters' links with each other).

In [ ]:
import urllib.request
import re
import string

In [ ]:
#Downloading entire Web Document (Raw Page Content)
def download_page(url):
    try:
        headers = {}
        headers['User-Agent'] = "Mozilla/5.0 (X11; Linux i686) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1312.27 Safari/537.17"
        req = urllib.request.Request(url, headers = headers)
        resp = urllib.request.urlopen(req)
        respData = str(resp.read())
        return respData
    except Exception as e:
        print(str(e))

In [ ]:
#Extract the title tag
def extract_info(page, start_tag, end_tag):
    start = page.find(start_tag)
    if start == -1:
        return '', -1
    else:
        end = page.find(end_tag, start + 1)
        n = len(start_tag)
        info = page[start + n : end]
        return info, end

In [ ]:
#Extract all the links
#Finding 'Next Link' on a given web page
def get_next_link(s):
    start_link = s.find("<a href")
    if start_link == -1:    #If no links are found then give an error!
        end_quote = 0
        link = "no_links"
        return link, end_quote
    else:
        start_quote = s.find('"', start_link)
        end_quote = s.find('"',start_quote+1)
        link = str(s[start_quote+1:end_quote])
        return link, end_quote

#Getting all links with the help of 'get_next_links'
def get_all_links(page):
    links = []
    while True:
        link, end_link = get_next_link(page)
        if link == "no_links":
            break
        else:
            links.append(link)      #Append all the links in the list named 'Links'
            page = page[end_link:]
    return links 

In [ ]:
#Season Appearance
def extract_season(page):
    try:
        season_path = seed_page + get_next_link(page[page.find('First appearance'):])[0]
        season = extract_info(download_page(season_path), 'Season', ',')[0]
        return re.sub("[^0-9]", "", season)
    except:
        return -1

In [ ]:
#Remove all the HTML tags from a paragraph of text
def extract_pure_text(page):
    pure_text = (re.sub(r'<.+?>', '', page))
    return pure_text

In [ ]:
#Crawl Initiation
#Check for file type in URL so crawler does not crawl images and text files
def extension_scan(url):
    a = ['.png','.jpg','.jpeg','.gif','.tif','.txt']
    j = 0
    while j < (len(a)):
        if a[j] in url:
            #print("There!")
            flag2 = 1
            break
        else:
            #print("Not There!")
            flag2 = 0
            j = j+1
    #print(flag2)
    return flag2

In [ ]:
#URL parsing for incomplete or duplicate URLs
def url_parse(url):
    try:
        from urllib.parse import urlparse
    except ImportError:
        from urlparse import urlparse
    url = url  #.lower()    #Make it lower case
    s = urlparse(url)       #parse the given url
    seed_page_n = seed_page #.lower()       #Make it lower case
    #t = urlparse(seed_page_n)     #parse the seed page (reference page)
    i = 0
    flag = 0
    while i<=9:
        if url == "/":
            url = seed_page_n
            flag = 0  
        elif not s.scheme:
            url = "http://" + url
            flag = 0
        elif "#" in url:
            url = url[:url.find("#")]
            flag = 0
        elif "?" in url:
            url = url[:url.find("?")]
            flag = 0
        elif s.netloc == "":
            url = seed_page + s.path
            flag = 0
        elif "www" not in url:
            url = "www."[:7] + url[7:]
            flag = 0
        elif url[len(url)-1] == "/":
            url = url[:-1]
            flag = 0     
        else:
            url = url
            flag = 0
            break
        
        i = i+1
        s = urlparse(url)
    return(url, flag)

In [ ]:
def find_relations(page, source_name, start_tag, end_tag, nodes, links, degree, visited):
    start = page.find(start_tag)
    end = page[start:].find(end_tag)
    i = start
    while i < start+end:
        if start == -1 or end == -1:
            break
        link, j = get_next_link(page[i:])        
        name = extract_info(page[i+j:], ">", "<")[0]
        flag = True
        for node in nodes:
            if node['link'] == url_parse(link)[0]:
                flag = False
        
        relation, k = extract_info(page[i+j:], "(", ")")
        relation = extract_pure_text(relation)
        
        if relation == "":
            break
        elif k != -1:
            i += k+j
            if flag:
                nodes += [{'link': url_parse(link)[0], 'name': name, 'degree': degree+1}]
            if link not in visited and link != "no_links":
                links += [{'source': source_name, 'target': name, 'relation': relation}]
                visited += [link]
        else:
            i += j

def find_people(page, source_name, start_tag, end_tag, relation, nodes, links, degree, visited):
    start = page.find(start_tag)
    end = page[start:].find(end_tag)
    i = start
    while i < start+end:
        if start == -1 or end == -1:
            break
        link, j = get_next_link(page[i:])
        name, k = extract_info(page[i+j:], ">", "<")
        flag = True
        for node in nodes:
            if node['link'] == url_parse(link)[0]:
                flag = False
                
        i += k+j
        if flag:
            nodes += [{'link': url_parse(link)[0], 'name': name, 'degree': degree+1}]
        if link not in visited and link != "no_links":
            links += [{'source': source_name, 'target': name, 'relation': relation}]
            visited += [link]

In [ ]:
#Defining pages
starting_page = "http://gossipgirl.wikia.com/wiki/Serena_van_der_Woodsen"
seed_page = "http://www.gossipgirl.wikia.com"  #Crawling the English Wikipedia

char_dict = {}
links = []
visited = []
iterator=0

# URL
char_dict['link'] = starting_page
# Degree
char_dict['degree'] = 0
degree = 0
# Name
page = download_page(starting_page)
char_dict['name'] = extract_info(page, r'<h1 class="page-header__title">', "</h1>")[0]
nodes = [char_dict]
visited += [char_dict['link']]

while(degree <= 1):
    page = download_page(nodes[iterator]['link'])
    name = nodes[iterator]['name']
    # Extract season
    nodes[iterator]['season'] = extract_season(page)
    # Define degree
    degree=nodes[iterator]['degree']
    
    # Find relationships
    try:
        page=page[:extract_info(page, "<span>Relationships</span>", "Series information</h2>")[1]]
        # Extract for family
        find_relations(page, name, "<b>Family</b></h3>", "<b>Romances</b></h3>", nodes, links, degree, visited)
        # Extract for Romances
        find_relations(page, name, "<b>Romances</b></h3>", "<b>Friends</b></h3>", nodes, links, degree, visited)
        # Extract for friends
        find_people(page, name, "<b>Friends</b></h3>", "<b>Enemies</b></h3>", 'Friend', nodes, links, degree, visited)
        # Extract for Enemies
        find_people(page, name, "<b>Enemies</b></h3>", "Series information</h2>", 'Enemy', nodes, links, degree, visited)
    except:
        break
    iterator += 1

In [ ]:
nodes

In [ ]:
links

In [ ]:
len(nodes)

In [ ]:
len(links)